# Python Intro - Final Exercise


In [2]:
import pyodbc
import pandas as pd
import numpy as np

In [3]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [118]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 
courses = pd.read_sql("SELECT * FROM Courses;",conn) 
departments = pd.read_sql("SELECT * FROM Departments;",conn) 
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn)
print(students)
print(teachers)
print(courses)
print(departments)
print(classrooms)

     StudentId FirstName   LastName Gender
0          162    Elmira    Holston      F
1          154    Sierra  Van Vorst      F
2           28     Mycah    Bingham      F
3           54      Alia      Jones      F
4           31    Chanda   Fillmore      F
..         ...       ...        ...    ...
275        246   Preston    Sprouse      M
276         55   Dmitriy       Gage      M
277        218    Skyler     Norris      M
278        269     Tyler    Raymond      M
279        228     Dylan      Davis      M

[280 rows x 4 columns]
    TeacherId  FirstName      LastName Gender
0           1     Kaylea      Monzingo      F
1           2     Amanda       Dworkin      F
2           3     Jordyn         Baron      F
3           4  Katherine        Dibari      F
4           5    Heather       Metcalf      F
5           6    Madison         Batan      F
6           7    Cortnee  Kinderknecht      F
7           8   Caroline       Knuesel      F
8           9   Katerina         Young      F


# Questions
 
 
## Q1. Count the number of students on each department

In [41]:
result = pd.merge(courses,departments , on="DepartmentId" )
result = pd.merge(result,classrooms , on="CourseId" )
result = result.drop_duplicates(subset=['DepartmentId','StudentId'], keep='first')
result = result.groupby(['DepartmentName']).agg({"StudentId":['nunique']})
print(result)

               StudentId
                 nunique
DepartmentName          
Arts                 137
English               84
Science              200
Sport                 81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [77]:
result = pd.merge(courses,classrooms , on="CourseId"  )
result = result[(result.DepartmentId == 1)]
result1 = result.groupby(['CourseName']).agg({"StudentId":['count']})
print(result1)
result2 = result.drop_duplicates(subset=['StudentId'], keep='first')["StudentId"].count()
print("the number of students in the department"  ,result2 )
      

                     StudentId
                         count
CourseName                    
Advanced English            24
English Begginers           32
Proffesional English        38
the number of students in the department 84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [138]:
result = pd.merge(courses,classrooms , on="CourseId"  )
result = result[(result.DepartmentId == 2)]
result = result.groupby(['CourseName']).agg({"StudentId":['count']})
result['classtype'] = 'big'
result.loc[(result.iloc[:,0]<=22) , 'classtype']='small'
result1 = result.groupby(['classtype']).agg({"classtype":['count']})
print(result1)




          classtype
              count
classtype          
big              10
small             6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [121]:
print(students.groupby(['Gender']).agg({"StudentId":['count']}))





       StudentId
           count
Gender          
F            165
M            115


## Q5. For which courses the percentage of male/female students is over 70%?

In [223]:
result = pd.merge(courses,classrooms , on="CourseId"  )
result = pd.merge(result,students , on="StudentId"  )
a = result.groupby(['CourseName']).agg(total=("StudentId", "count"))
m = result[(result.Gender == "M")]
f = result[(result.Gender == "F")]
m = m.groupby(['CourseName']).agg(males=("StudentId", "count"))
f = f.groupby(['CourseName']).agg(females=("StudentId", "count"))
resultall = pd.merge(m, f,  on="CourseName" ,how = 'inner')
resultall = pd.merge(resultall, a,  on="CourseName" ,how = 'inner')
resultall['pm'] = resultall['males'] / resultall ['total'] > 0.7
resultall['pf'] = resultall['females'] / resultall ['total'] > 0.7
resultall = resultall[(resultall.pf == True) | (resultall.pm == True) ]
print(resultall.iloc[:,0:3])

            males  females  total
CourseName                       
Sculpture       7       17     24
Tenis           5       13     18


## Q6. For each department, how many students passed with a grades over 80?

In [231]:
result = pd.merge(courses,departments , on="DepartmentId" )
result = pd.merge(result,classrooms , on="CourseId"  )
result = pd.merge(result,students , on="StudentId"  )
result = result[(result.Degree >80 )]
result = result.groupby(['DepartmentName']).agg(over80=("StudentId", "nunique"))
print(result)


                over80
DepartmentName        
Arts                61
English             40
Science            122
Sport               39


## Q7. For each department, how many students passed with a grades under 60?

In [232]:
result = pd.merge(courses,departments , on="DepartmentId" )
result = pd.merge(result,classrooms , on="CourseId"  )
result = pd.merge(result,students , on="StudentId"  )
result = result[(result.Degree <60 )]
result = result.groupby(['DepartmentName']).agg(over80=("StudentId", "nunique"))
print(result)


                over80
DepartmentName        
Arts                 3
English              2
Science              7
Sport                1


## Q8. Rate the teachers by their average student's grades (in descending order).

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).